# Complete pipeline

1. We create a simple environment in Flatland that has only a single agent.  The agent is given a starting position and a goal.
2. This information is fed to a Clingo encoding, which produces a path for the agent from the start to the goal.
3. We return this path information to Flatland in a manner which can be interpreted and visualized.

## Flatland environment generation

This is Flatland code, not my own.

In [14]:
# import packages
import numpy as np

from flatland.envs.rail_env import RailEnv, RailEnvActions
from flatland.envs.rail_generators import rail_from_manual_specifications_generator
from flatland.envs.rail_generators import rail_from_grid_transition_map
from flatland.envs.rail_generators import sparse_rail_generator
from flatland.envs.observations import TreeObsForRailEnv

from flatland.core.grid.rail_env_grid import RailEnvTransitions
from flatland.core.transition_map import GridTransitionMap

In [66]:
def make_environment():
    """create an environment by specifying the track type of each cell"""
    transitions = RailEnvTransitions()
    cells = transitions.transition_list
    
    # rail map: track types by ID values
    rail_map = np.array(
        [[0] + [16386] + [256]]
        + [[4] + [3089] +[256]]
        , dtype=np.uint16)

    # rail: transitions based on track types
    rail = GridTransitionMap(width=rail_map.shape[1], height=rail_map.shape[0], transitions=transitions)
    
    # optionals
    rail.grid = rail_map
    city_positions = [(0,1), (3,0)]
    train_stations = [[((0,1),0)], [((3,0),0)],]
    city_orientations = [0,0]
    agents_hints = {
        'city_positions': city_positions,
        'train_stations': train_stations,
        'city_orientations': city_orientations
    }
    optionals = {'agents_hints': agents_hints}
    return rail, rail_map, optionals

In [67]:
rail, rail_map, optionals = make_environment()
rail_gen = rail_from_grid_transition_map(rail_map=rail, optionals=optionals)
hand_crafted_env = RailEnv(width=3, height=2, rail_generator=rail_gen, number_of_agents=1, obs_builder_object=TreeObsForRailEnv(max_depth=0))


### Example output of Flatland's `rail_map`

In [125]:
rail_map


array([[    0, 16386,   256],
       [    4,  3089,   256]], dtype=uint16)

## Clingo pathfinding encoding

In [58]:
from pathlib import Path

### `clingo_map` class

In [137]:
class clingo_map():
    """an alternative representation of a Flatland map that clingo can process"""
    
    def __init__(self, rail_map):
        self.clingo_str = ""
        self.mapping = {}
        
        row_num = len(rail_map) - 1
        for row in rail_map:
            for col,cval in enumerate(row):
                self.clingo_str += "cell(({},{}), {}). ".format(col,row_num,cval)
                self.mapping[(col,row_num)] = cval
            row_num -= 1


In [143]:
env = clingo_map(rail_map)
print(env.clingo_str) # take a look


cell((0,1), 0). cell((1,1), 16386). cell((2,1), 256). cell((0,0), 4). cell((1,0), 3089). cell((2,0), 256). 


In [139]:
import clingo

def on_model(m):
    """print results upon solving"""
    print(m)

In [140]:
# clingo API
ctl = clingo.Control()

# load the map into clingo
ctl.add("base", [], env.clingo_str)

# load the types.lp reference file
types = Path('../attempt_05_full/types.lp').read_text()
#ctl.add("base", [], types)

# load the encoding
#encoding = Path('../attempt_05_full/encoding.lp').read_text() 
#ctl.add("base", [], encoding)

# ground & solve
ctl.ground([("base", [])])
ctl.solve(on_model=on_model)

cell((0,1),0) cell((1,1),16386) cell((2,1),256) cell((0,0),4) cell((1,0),3089) cell((2,0),256)


SolveResult(1)

**Expected result**

`at(cell(0,0),0,e) at(cell(1,0),1,e) at(cell(1,1),2,n) at(cell(2,1),3,e)`

In [141]:
env.mapping

{(0, 1): 0, (1, 1): 16386, (2, 1): 256, (0, 0): 4, (1, 0): 3089, (2, 0): 256}

### Translate solution → actions

In [150]:
# translate clingo solution into moves
switch_ids = [37408,17411,32872,3089,49186,1097,34864,5633,38433,50211,33897,35889,38505,52275,20994,16458,2136,6672]

def is_switch(loc, env): #env.mapping
    """
    given a location and its environment, determine whether the cell has a switch
    """
    return env[loc] in switch_ids

def next_move(dirs, locs, env):
    """
    given:
        * a current cardinal direction and the following one, 
        * a current location and the following one
        * an environment
    return what move the train should make
    """
    if locs[0] == locs[1]:
        return "STOP_MOVING"
    elif dirs[0] == dirs[1]:
        return "MOVE_FORWARD"
    #elif 


In [148]:
is_switch((1,0),env.mapping)

True

## Flatland visualization

In [3]:
# code here

In [10]:
%%clingo 0

p(1). p(2). p(3).
{ holds(q(X)) : p(X) }.
holds(a) :- holds(q(1)).
#show.
#show X : holds(X).

clingo version 5.6.2
Reading from stdin
Solving...
Answer: 1

Answer: 2
q(2)
Answer: 3
q(3)
Answer: 4
q(2) q(3)
Answer: 5
q(1) a
Answer: 6
q(1) q(3) a
Answer: 7
q(1) q(2) a
Answer: 8
q(1) q(2) q(3) a
SATISFIABLE

Models       : 8
Calls        : 1
Time         : 0.008s (Solving: 0.00s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.002s
